In [ ]:
!pip install accelerate -U
!pip install transformers[torch]

# Final results at the end of the notebook

# Multi-Label Classification

In [ ]:
# HERE WE ARE TRAINING A MULTI-LABEL CLASSIFIER TO GET APPROPRIATE CATEGORIES
# FOR THE GIVEN SENTENCE USING TRANSFER LEARNING ON BERT TRANSFORMER

# DUE TO THE LARGE NUMBER OF CATEGORIES AND HOW MUCH MORE BETTER BERT IS AT
# "UNDERSTANDING" CONTEXT I'VE AVOIDED USING SIMPLER MODELS LIKE SUPPORT VECTOR MACHINES

import pandas as pd
from sklearn.model_selection import train_test_split

# getting the dataset ready for training
df = pd.read_csv('./category.csv')
category_columns =  ['garage service', 'ease of booking', 'value for money', 'location',
       'length of fitting', 'change of date', 'tyre quality', 'wait time',
       'delivery punctuality', 'mobile fitter', 'advisor/agent service',
       'advisoragent service', 'extra charges', 'damage', 'balancing',
       'facilities', 'change of time', 'booking confusion', 'late notice',
       'discounts', 'refund not actioned', 'refund timescale',
       "mobile fitter didn't arrive", 'discount not applied',
       'tyre agedot code', 'failed payment', 'incorrect tyres sent',
       'call wait time', 'refund', 'no stock', 'response time',
       'tyre age/dot code']


X = df['sentence'].tolist()
y = df[category_columns].values.tolist()
# splitting
X_train, X_eval, y_train, y_eval = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
df.columns

Index(['Unnamed: 0', 'sentence', 'garage service', 'ease of booking',
       'value for money', 'location', 'length of fitting', 'change of date',
       'tyre quality', 'wait time', 'delivery punctuality', 'mobile fitter',
       'advisor/agent service', 'advisoragent service', 'extra charges',
       'damage', 'balancing', 'facilities', 'change of time',
       'booking confusion', 'late notice', 'discounts', 'refund not actioned',
       'refund timescale', 'mobile fitter didn't arrive',
       'discount not applied', 'tyre agedot code', 'failed payment',
       'incorrect tyres sent', 'call wait time', 'refund', 'no stock',
       'response time', 'tyre age/dot code'],
      dtype='object')

In [ ]:
from transformers import BertTokenizer

# using tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

train_encodings = tokenizer(X_train, truncation=True, padding=True, max_length=128)
eval_encodings = tokenizer(X_eval, truncation=True, padding=True, max_length=128)

In [ ]:
import torch
from torch.utils.data import Dataset

class MultiLabelDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx], dtype=torch.float)
        return item

    def __len__(self):
        return len(self.labels)

# Create the dataset
train_datasetz = MultiLabelDataset(train_encodings, y_train)
eval_datasetz = MultiLabelDataset(eval_encodings, y_eval)

In [ ]:
from transformers import BertForSequenceClassification, Trainer, TrainingArguments

# Load a pre-trained BERT model for sequence classification with a specific number of labels
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=len(category_columns), problem_type="multi_label_classification")

# Define training arguments
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    warmup_steps=10,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
    evaluation_strategy="epoch"
)

# Define the trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_datasetz,
    eval_dataset=eval_datasetz
)

# Train the model
trainer.train()


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,0.081100,0.085537
2,0.070200,0.067616
3,0.065200,0.062048


TrainOutput(global_step=4875, training_loss=0.08302728243974539, metrics={'train_runtime': 795.1896, 'train_samples_per_second': 24.522, 'train_steps_per_second': 6.131, 'total_flos': 1283011891200000.0, 'train_loss': 0.08302728243974539, 'epoch': 3.0})

In [73]:
from transformers import BertForSequenceClassification

new_sentences = ["Good price, great service"]
new_encodings = tokenizer(new_sentences, truncation=True, padding=True, max_length=128)

# testing
model = BertForSequenceClassification.from_pretrained('./saved_model/category')
trainer = Trainer(model)

new_dataset = MultiLabelDataset(new_encodings, [[0]*len(category_columns)])
predictions = trainer.predict(new_dataset)

# convert to binary predictions to narrow down categories
predicted_labels = (torch.sigmoid(torch.tensor(predictions[0])) > 0.4).int()

# map predictions
predicted_categories = [category_columns[i] for i in range(len(category_columns)) if predicted_labels[0][i] == 1]

print("Predicted categories:", predicted_categories)

output_dir = "./saved_model/category"

# # Save the model weights and tokenizer
model.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)

Predicted categories: ['value for money']


('./saved_model/category/tokenizer_config.json',
 './saved_model/category/special_tokens_map.json',
 './saved_model/category/vocab.txt',
 './saved_model/category/added_tokens.json')

# Sentiment Analyser

In [41]:
# WHILE TRAINING THE SENTIMENT ANALYSER I WANT TO PASS BOTH THE CATEGORY AND THE
# SENTENCE BECAUSE THAT'S HOW WE KNOW IF THE SUB-SENTIMENT ANALYSER WORKS
# THE REST IS ALMOST THE SAME AS THE PREVIOUS ONE

import pandas as pd

df = pd.read_csv('./sentiment.csv')


#  combine 'sentence' and 'category' columns
df['input'] = df['sentence'] + " [CATEGORY] " + df['category']

# map sentiments to numerical values
sentiment_mapping = {"positive": 1, "negative": 0}
df['label'] = df['sentiment'].map(sentiment_mapping)

# separate features and labels
X = df['input'].tolist()
y = df['label'].tolist()
X_train, X_eval, y_train, y_eval = train_test_split(X, y, test_size=0.2, random_state=42)

In [42]:
from transformers import BertTokenizer

# tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

train_encodings = tokenizer(X_train, truncation=True, padding=True, max_length=128)
eval_encodings = tokenizer(X_eval, truncation=True, padding=True, max_length=128)

In [43]:
import torch
from torch.utils.data import Dataset

class SentimentDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx], dtype=torch.long)
        return item

    def __len__(self):
        return len(self.labels)

train_datasetz = SentimentDataset(train_encodings, y_train)
eval_datasetz = SentimentDataset(eval_encodings, y_eval)

In [44]:
from transformers import BertForSequenceClassification, Trainer, TrainingArguments

# load pretrained model
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)

# arguments
training_args = TrainingArguments(
    output_dir='./sentiment/results',
    num_train_epochs=3,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    warmup_steps=10,
    weight_decay=0.01,
    logging_dir='./sentiment/logs',
    logging_steps=10,
    evaluation_strategy="epoch"
)

# trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_datasetz,
    eval_dataset=eval_datasetz
)

# train the model
trainer.train()

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,0.367500,0.284297
2,0.198400,0.241778
3,0.098800,0.241979


TrainOutput(global_step=8700, training_loss=0.24669371295757805, metrics={'train_runtime': 1587.0368, 'train_samples_per_second': 21.928, 'train_steps_per_second': 5.482, 'total_flos': 2289066181632000.0, 'train_loss': 0.24669371295757805, 'epoch': 3.0})

In [45]:
new_sentences = ["Easy process but problems with booking a fitting appointment. The website made it easy to find the correct tyre specifications for my vehicle, and to place the order. I was offered a number of fitting options and chose ATS tyres, then selected a time slot to suit my convenience. Unfortunately the time slot I selected was not available at ATS and I had to wait a further two days for a convenient time, at which point the tyres were fitted quickly and professionally. Would have given five stars apart from being offered a fitting time that they couldn't honour."]
new_categories = ["change of time", "ease of booking", "length of fitting", "garage service"]

predicted_sentiments = []

for category in new_categories:
    # create new input for the current category
    new_inputs = [f"{sentence} [CATEGORY] {category}" for sentence in new_sentences]

    # tokenize the new inputs
    new_encodings = tokenizer(new_inputs, truncation=True, padding=True, max_length=128)

    # convert to dataset
    new_dataset = SentimentDataset(new_encodings, [0] * len(new_inputs))  # Labels are dummy values

    # get predictions
    predictions = trainer.predict(new_dataset)

    # 0:negative,1:positive
    predicted_sentiments.append(torch.argmax(torch.tensor(predictions[0]), dim=1).tolist())

# map numerical predictions back to sentiment labels
reverse_sentiment_mapping = {v: k for k, v in sentiment_mapping.items()}
predicted_labels = [[reverse_sentiment_mapping[sentiment] for sentiment in sentiments] for sentiments in predicted_sentiments]

print("Predicted sentiments for each category:")
for category, sentiments in zip(new_categories, predicted_labels):
    print(f"{category}: {sentiments}")

output_dir = "./saved_model/sentiment"

# save the model weights and tokenizer
model.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)

Predicted sentiments for each category:
change of time: ['negative']
ease of booking: ['positive']
length of fitting: ['positive']
garage service: ['positive']


('./saved_model/sentiment/tokenizer_config.json',
 './saved_model/sentiment/special_tokens_map.json',
 './saved_model/sentiment/vocab.txt',
 './saved_model/sentiment/added_tokens.json')

# Combining Models


In [85]:
# NOW THAT WE HAVE TRAINED OUR TWO MODELS AND HAVE THE APPROPRIATE WEIGHTS READY
# WE CAN COMBINE THEM TOGETHER
# HERE WE PASS THE INPUT STRING
  # THROUGH THE CATEGORY MULTI-LABEL CLASSIFIER FIRST
  # WE ITERATIVELY PASS THE OUTPUT ALONG WITH THE INPUT STRING TO THE SENTIMENT ANALYSER
  # AND GET THE FINAL OUTPUT IN A DICTIONARY

from transformers import BertForSequenceClassification, BertTokenizer
import torch

model1 = BertForSequenceClassification.from_pretrained('./saved_model/category')
model2 = BertForSequenceClassification.from_pretrained('./saved_model/sentiment')
trainer1 = Trainer(model1)
trainer2 = Trainer(model2)
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
sentiment_mapping = {"positive": 1, "negative": 0}

# Define categories
category_columns = ['garage service', 'ease of booking', 'value for money', 'location',
       'length of fitting', 'change of date', 'tyre quality', 'wait time',
       'delivery punctuality', 'mobile fitter', 'advisor/agent service',
       'advisoragent service', 'extra charges', 'damage', 'balancing',
       'facilities', 'change of time', 'booking confusion', 'late notice',
       'discounts', 'refund not actioned', 'refund timescale',
       "mobile fitter didn't arrive", 'discount not applied',
       'tyre agedot code', 'failed payment', 'incorrect tyres sent',
       'call wait time', 'refund', 'no stock', 'response time',
       'tyre age/dot code']

def predict_sentiment(input_string):
    new_encodings = tokenizer(input_string, truncation=True, padding=True, max_length=128)

    new_dataset1 = MultiLabelDataset(new_encodings, [[0]*len(category_columns)])
    predictions = trainer1.predict(new_dataset1)

    # convert to binary predictions to narrow down categories
    predicted_labels = (torch.sigmoid(torch.tensor(predictions[0])) > 0.4).int()
    #    map predictions
    predicted_categories = [category_columns[i] for i in range(len(category_columns)) if predicted_labels[0][i] == 1]
    if not predicted_categories:
      return None

    predicted_sentiments = []

    for category in predicted_categories:
      # create new input for the current category
      new_inputs = [f"{sentence} [CATEGORY] {category}" for sentence in input_string]

      # tokenize the new inputs
      new_encodings = tokenizer(new_inputs, truncation=True, padding=True, max_length=128)
      new_dataset = SentimentDataset(new_encodings, [0] * len(new_inputs))  # labels are dummy values

      # get predictions
      predictions = trainer2.predict(new_dataset)

      # 0:negative,1:positive
      predicted_sentiments.append(torch.argmax(torch.tensor(predictions[0]), dim=1).tolist())

    # map numerical predictions back to sentiment labels
    reverse_sentiment_mapping = {v: k for k, v in sentiment_mapping.items()}
    predicted_labels = [[reverse_sentiment_mapping[sentiment] for sentiment in sentiments] for sentiments in predicted_sentiments]

    print("Predicted sentiments for each category:")
    output = {}
    for category, sentiments in zip(predicted_categories, predicted_labels):
      output[category] = sentiments

    return output

# THE INPUT STRING
input_string = [["Good price, great service"],["Was messed around quite a lot - times rearranged due to tyres not being delivered, then put back to the original appointment - without cancellung the changed one. Will go directly through the garage next time. [REDACTED] messed me about too much for the same price as the garage would have given without them."],["Excellent service and saved a few pounds Will definitely be using [REDACTED] again."],["Competitive prices and consistently excellent. This has been the case for many years. No problems with the tyres being sent on time to the fitters or the fitters themselves."]]

for st in input_string:
  print(st)
  predicted_sentiments = predict_sentiment(st)
  print(predicted_sentiments)
  print("\n")

['Good price, great service']


Predicted sentiments for each category:
{'value for money': ['positive']}


['Was messed around quite a lot - times rearranged due to tyres not being delivered, then put back to the original appointment - without cancellung the changed one. Will go directly through the garage next time. [REDACTED] messed me about too much for the same price as the garage would have given without them.']


Predicted sentiments for each category:
{'value for money': ['negative'], 'change of date': ['negative'], 'delivery punctuality': ['negative']}


['Excellent service and saved a few pounds Will definitely be using [REDACTED] again.']


Predicted sentiments for each category:
{'value for money': ['positive']}


['Competitive prices and consistently excellent. This has been the case for many years. No problems with the tyres being sent on time to the fitters or the fitters themselves.']


Predicted sentiments for each category:
{'value for money': ['positive'], 'delivery punctuality': ['positive']}




In [86]:
!zip -r /content/saved_model.zip /content/saved_model

  adding: content/saved_model/ (stored 0%)
  adding: content/saved_model/category/ (stored 0%)
  adding: content/saved_model/category/config.json (deflated 65%)
  adding: content/saved_model/category/model.safetensors (deflated 7%)
  adding: content/saved_model/category/special_tokens_map.json (deflated 42%)
  adding: content/saved_model/category/vocab.txt (deflated 53%)
  adding: content/saved_model/category/tokenizer_config.json (deflated 75%)
  adding: content/saved_model/sentiment/ (stored 0%)
  adding: content/saved_model/sentiment/config.json (deflated 49%)
  adding: content/saved_model/sentiment/model.safetensors (deflated 7%)
  adding: content/saved_model/sentiment/special_tokens_map.json (deflated 42%)
  adding: content/saved_model/sentiment/vocab.txt (deflated 53%)
  adding: content/saved_model/sentiment/tokenizer_config.json (deflated 75%)
